In [2]:
from subprocess import check_output
print(check_output(["ls", "../input/instacart-market-basket-analysis"]).decode("utf8"))

aisles.csv.zip
departments.csv.zip
order_products__prior.csv.zip
order_products__train.csv.zip
orders.csv.zip
products.csv.zip
sample_submission.csv.zip



In [3]:
import os
import zipfile

for dirname, _, filenames in os.walk('/kaggle/input/instacart-market-basket-analysis/'):
    for filename in filenames:        
        archive = zipfile.ZipFile(os.path.join(dirname, filename), mode='r')
        archive.extractall(path="/kaggle/working")
        archive.close()

print(check_output(["ls", "../working"]).decode("utf8"))

__MACOSX
__notebook_source__.ipynb
aisles.csv
departments.csv
order_products__prior.csv
order_products__train.csv
orders.csv
products.csv
sample_submission.csv



In [4]:
import pandas as pd

order_products_train = pd.read_csv("../working/order_products__train.csv")
order_products_prior = pd.read_csv("../working/order_products__prior.csv")
orders = pd.read_csv("../working/orders.csv")
products = pd.read_csv("../working/products.csv")
aisles = pd.read_csv("../working/aisles.csv")
departments = pd.read_csv("../working/departments.csv")
sample_submission = pd.read_csv("../working/sample_submission.csv")

print('*** df 생성 완료 ***')

*** df 생성 완료 ***


In [5]:
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


In [4]:
orders[orders['order_id']==1]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
1868044,1,112108,train,4,4,10,9.0


In [7]:
order_products_train[order_products_train['order_id']==1]

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
5,1,13176,6,0
6,1,47209,7,0
7,1,22035,8,1


In [8]:
orders.eval_set.unique()

array(['prior', 'train', 'test'], dtype=object)

In [5]:
filter_orders = orders[(orders['eval_set'] == 'train') | (orders['eval_set'] == 'prior')]
filter_orders.eval_set.unique()

array(['prior', 'train'], dtype=object)

In [10]:
print(filter_orders.order_id.nunique())
order_products_train.order_id.nunique() + order_products_prior.order_id.nunique() 

3346083


3346083

## Merge data set : products + aisles + departments

In [6]:
merged_products = pd.merge(products, aisles, on='aisle_id', how='left')
merged_products = pd.merge(merged_products, departments, on='department_id', how='left')
merged_products = merged_products.reindex(columns=['product_id','product_name','aisle_id','aisle','department_id','department'])
merged_products.head()

,product_id,product_name,aisle_id,aisle,department_id,department
0,1,Chocolate Sandwich Cookies,61,cookies cakes,19,snacks
1,2,All-Seasons Salt,104,spices seasonings,13,pantry
2,3,Robust Golden Unsweetened Oolong Tea,94,tea,7,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,frozen meals,1,frozen
4,5,Green Chile Anytime Sauce,5,marinades meat preparation,13,pantry


## Merge data set : order_products_train + order_products_prior

In [6]:
train_prior = pd.concat([order_products_train, order_products_prior])
train_prior

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
32434484,3421083,39678,6,1
32434485,3421083,11352,7,0
32434486,3421083,4600,8,0
32434487,3421083,24852,9,1


## Merge data set : train_prior + merged_products

In [7]:
train_prior = pd.merge(train_prior, merged_products, on='product_id', how='left')
train_prior

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,aisle,department_id,department
0,1,49302,1,1,Bulgarian Yogurt,120,yogurt,16,dairy eggs
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,other creams cheeses,16,dairy eggs
2,1,10246,3,0,Organic Celery Hearts,83,fresh vegetables,4,produce
3,1,49683,4,0,Cucumber Kirby,83,fresh vegetables,4,produce
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,canned meat seafood,15,canned goods
...,...,...,...,...,...,...,...,...,...
33819101,3421083,39678,6,1,Free & Clear Natural Dishwasher Detergent,74,dish detergents,17,household
33819102,3421083,11352,7,0,Organic Mini Sandwich Crackers Peanut Butter,78,crackers,19,snacks
33819103,3421083,4600,8,0,All Natural French Toast Sticks,52,frozen breakfast,1,frozen
33819104,3421083,24852,9,1,Banana,24,fresh fruits,4,produce


In [8]:
del merged_products

## Merge data set : filter_orders + merged_products

In [15]:
print(filter_orders.shape)
train_prior.shape

(3346083, 7)


(33819106, 9)

In [9]:
merged_df = train_prior.merge(filter_orders, left_on='order_id', right_on='order_id', how='outer')
merged_df

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,aisle,department_id,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,Bulgarian Yogurt,120,yogurt,16,dairy eggs,112108,train,4,4,10,9.0
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,other creams cheeses,16,dairy eggs,112108,train,4,4,10,9.0
2,1,10246,3,0,Organic Celery Hearts,83,fresh vegetables,4,produce,112108,train,4,4,10,9.0
3,1,49683,4,0,Cucumber Kirby,83,fresh vegetables,4,produce,112108,train,4,4,10,9.0
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,canned meat seafood,15,canned goods,112108,train,4,4,10,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33819101,3421083,39678,6,1,Free & Clear Natural Dishwasher Detergent,74,dish detergents,17,household,25247,prior,24,2,6,21.0
33819102,3421083,11352,7,0,Organic Mini Sandwich Crackers Peanut Butter,78,crackers,19,snacks,25247,prior,24,2,6,21.0
33819103,3421083,4600,8,0,All Natural French Toast Sticks,52,frozen breakfast,1,frozen,25247,prior,24,2,6,21.0
33819104,3421083,24852,9,1,Banana,24,fresh fruits,4,produce,25247,prior,24,2,6,21.0


In [10]:
del order_products_train
del order_products_prior
del orders
del products
del aisles
del departments
del sample_submission

In [ ]:
print(merged_df.isnull().sum())
merged_df.info()

In [10]:
merged_df.order_id.nunique()

3346083

## 불필요한 칼럼 제거 

In [ ]:
filtered = merged_df.drop(['add_to_cart_order','reordered','user_id','eval_set','order_number','order_dow','order_hour_of_day','days_since_prior_order','product_id','product_name','aisle_id','aisle','department_id'], axis = 1)
filtered

In [1]:
onehot_filtered = pd.get_dummies(filtered)
onehot_filtered

NameError: name 'pd' is not defined

In [ ]:
onehot_filtered.groupby(by=['order_id']).sum()
onehot_filtered